# CHATBOT PARA CONSULTAS DE TECNICOS EN INSTALACIONES DE SISTEMA SONAR - PARTE 1
En esta primera parte realizaremos la extraccion de los datos de 3 pdfs donde contiene la información necesaria para el reentranamiento de los datos y generaremos las preguntas y respuestas que posteriormente guardaremos en un JSON.

In [ ]:
# instalamos las librerias necesarias
!pip install langchain pypdf  langchain_experimental langchain_openai langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32


In [ ]:
# preparo los documentos en una lista con las rutas y en que pagina inicia el documento porque el resto contine informacion no relevante(Titulos e indices)

datosPdfs = [
    {
        "ruta": './EtFu1051_C1.pdf',
        "pagini":3
    },
    {
        "ruta": './MaIn0362_C1.pdf',
        "pagini":6
    },
    {
        "ruta": './MaUs0240_C1.pdf',
        "pagini":4
    }

]


print(datosPdfs[0]['ruta'])
print(type(datosPdfs))

./EtFu1051_C1.pdf
<class 'list'>


**Vamos a separar el texto en encabezados porque los documentos vienen separados por ellos**

In [ ]:
import re
from langchain.document_loaders import PyPDFLoader

# utilizamos expresiones regulares para detectar encabezados
expresion_regular = re.compile(r"(?=\n?\d+(\.\d+)*\s+[A-ZÁÉÍÓÚÑ]|CAPÍTULO\s+\d+)", re.MULTILINE)


def split_por_cabeceras(text):
    headers = expresion_regular.split(text)
    return headers

def extraer_texto_pdfs(list_pdfs):

  # creamos una lista donde almacenaremos todos los chunks de los documentos para posteriormente generar las preguntas
  chunks_list_sonar = []

  # realizamos un bucle para iterar entre los documentos
  for pdf in list_pdfs:



    # cargamos el contenido
    loader = PyPDFLoader(pdf['ruta'])
    paginas = loader.load()

    paginas = paginas[pdf['pagini'] - 1:]

    # obtenemos todo el texto del PDF
    full_text = "\n".join([page.page_content for page in paginas])

    # separamos por encabezados
    secciones = split_por_cabeceras(full_text)


    # preprocesamos los datos para evitar valores nulos o secciones con poco contenido, tambien quitamos saltos de lineas innecesarios que ensucian el contenido
    for sec in secciones:
        # comprobamos que el valor nos None
        if sec is not None:
          # quitamos todas las secciones con menos de 80 caracteres
          if(len(sec) > 60):
            # reemplazamos los saltos de lineas por espacios
            sec = sec.replace("\n", " ")
            # lo añadimos a la lista de todas las secciones
            chunks_list_sonar.append(sec)


  return chunks_list_sonar


In [ ]:
chunks_list_sonar = extraer_texto_pdfs(datosPdfs)

## Generacion de preguntas y respuestas
Para esta parte vamos a usar los chunks que hemos extraido de los pdfs y vamos a utilizar el modelo de chatgpt 3.5-turbo, para que genere preguntas y respuestas, despues lo guardara en un JSON para usarlo en el fine tuning

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# configuramos el modelo, indicando cual vamos a usar y su key
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key="XXX")

# creamos una funcion que se encargara de ejecutar el modelo y mandarle el prompt base
def generar_qas(chunk):
    prompt = f"""Genera preguntas y respuestas basadas en el siguiente contenido:\n\n{chunk}\n\nFormato:\nPregunta: ...\nRespuesta: ..."""

    response = llm([HumanMessage(content=prompt)])
    return response.content

# ejecutamos la funcion iterando entre todos los chunks
qa_parejas = []
for doc in chunks_list_sonar:
    qas = generar_qas(doc)
    qa_parejas.append(qas)

# guardamos en un json
import json

with open("qa_dataset.json", "w", encoding="utf-8") as f:
    json.dump(qa_parejas, f, indent=4, ensure_ascii=False)

print("Dataset de QA generado y guardado.")


Dataset de QA generado y guardado.


In [ ]:
# observamos que el dataset esta mal estructurado lo mejoramos


file_path = "./qa_dataset.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Procesar el contenido del JSON para separar cada pregunta y respuesta en una fila
qa_list = []
for item in data:
    qa_pairs = item.split("\n\n")
    for pair in qa_pairs:
        if "Pregunta:" in pair and "Respuesta:" in pair:
            question, answer = pair.split("\nRespuesta: ", 1)
            question = question.replace("Pregunta: ", "").strip()
            answer = answer.strip()
            qa_list.append({"pregunta": question, "respuesta": answer})

# guardamos en un json
import json

with open("qa_dataset_mejorado.json", "w", encoding="utf-8") as f:
    json.dump(qa_list, f, indent=4, ensure_ascii=False)